In [1]:
import gzip
import os
import pandas as pd
import numpy as np

In [19]:
ga = 'GRCh38'
filepath = f'{ga}/appris_method.spade.gtf.gz'

In [20]:
f = gzip.open(filepath, 'rb')

gtf = [
    sorted([
        j.strip().split(' ')
        for j in line.decode('utf-8').strip().split('\t')[-1].split(';') # loop subfields in last field
    ])
    for line in f # loop lines
]

f.close()

In [21]:
gtf = [
    ([j1, j3, *[k.split(':') for k in j2[1].replace('"','').split(',')]])
    for j1,j2,j3 in gtf
]

In [22]:
# Some domains do not have evalue
gtf = [i if 'evalue' in list(zip(*i))[0] else i + [['evalue', np.nan]] for i in gtf]

# Sort fields
gtf = [sorted(i) for i in gtf]

In [23]:
gtf_rows = [list(zip(*i))[1] for i in gtf]
gtf_columns = list(zip(*gtf[0]))[0]

print(gtf_columns)

('evalue', 'gene_id', 'hmm_name', 'pep_end', 'pep_start', 'transcript_id')


In [24]:
dfgtf = pd.DataFrame(
    [list(zip(*i))[1] for i in gtf], 
    columns=('evalue', 'gene_id', 'hmm_name', 'pep_end', 'pep_start', 'transcript_id')
).astype({'evalue':'float64', 'pep_start':'int64','pep_end':'int64'})

dfgtf['gene_id'] = dfgtf['gene_id'].str.replace('"', '')
dfgtf['transcript_id'] = dfgtf['transcript_id'].str.replace('"', '')

In [8]:
dfgtf.dtypes

evalue           float64
gene_id           object
hmm_name          object
pep_end            int64
pep_start          int64
transcript_id     object
dtype: object

In [25]:
dfgtf

,evalue,gene_id,hmm_name,pep_end,pep_start,transcript_id
0,1.800000e-51,ENSG00000198967,7tm_4,305,31,ENST00000641002
1,2.900000e-120,ENSG00000162654,GBP,295,33,ENST00000355754
2,4.400000e-128,ENSG00000162654,GBP_C,594,298,ENST00000355754
3,2.500000e-73,ENSG00000117226,GBP_C,203,2,ENST00000461384
4,1.800000e-125,ENSG00000117226,GBP,279,19,ENST00000370481
...,...,...,...,...,...,...
168376,1.100000e-13,ENSG00000198786,Proton_antipo_N,121,65,ENST00000361567
168377,1.300000e-73,ENSG00000198786,Proton_antipo_M,418,134,ENST00000361567
168378,2.700000e-53,ENSG00000198786,NADH5_C,602,422,ENST00000361567
168379,9.200000e-26,ENSG00000198712,COX2_TM,83,1,ENST00000361739


In [26]:
dfgtf.to_feather(f'{ga}/appris_method.spade.feather')

In [27]:
dfgtf.to_csv(f'{ga}/appris_method.spade.csv')